In [1]:
%run "/home/jovyan/work/workspace/functions/create_spark_session.ipynb"

Spark Version: 3.5.0


In [2]:
%run "/home/jovyan/work/workspace/functions/paths.ipynb"

In [3]:
%run "/home/jovyan/work/workspace/functions/dates.ipynb"

In [4]:
f'{gold}/sales'

'/home/jovyan/work/lake/gold//sales'

In [5]:
!ls /home/jovyan/work/lake/gold

channel_group  sales  sales_fact  sales_location  sales_product  sales_time


In [6]:
sales = spark.read.format('delta').load(f'{gold}sales')

In [7]:
sales_location = spark.read.format('delta').load(f'{gold}sales_location')

In [8]:
sales_product = spark.read.format('delta').load(f'{gold}sales_product')

In [9]:
sales_time = spark.read.format('delta').load(f'{gold}sales_time')

In [10]:
channel_group = spark.read.format('delta').load(f'{gold}channel_group')

In [11]:
sales_fact = spark.read.format('delta').load(f'{gold}sales_fact')

In [12]:
sales.createOrReplaceTempView('sales')
sales_product.createOrReplaceTempView('sales_product')
sales_time.createOrReplaceTempView('sales_time')
channel_group.createOrReplaceTempView('channel_group')
sales_fact.createOrReplaceTempView('sales_fact')

#### Create queries/scripts that can answer following business questions using the data structure created:
Expected result: Data processes and results.

What are the Top 3 Trade Groups (TRADE_GROUP_DESC) for each Region (Btlr_Org_LVL_C_Desc) in sales ($ Volume)?


In [13]:
sales.columns

['dtregister',
 'cdbrand',
 'dsbrand',
 'dsregion',
 'cdgroup',
 'tpgroup',
 'dsgroup',
 'dschannel',
 'dspack',
 'dscatpack',
 'dsnmpack',
 'qtvolume',
 'dsyear',
 'dsmonth',
 'dsquarter',
 'dtload']

In [14]:
top3 = spark.sql('''
WITH sales_per_region as (

SELECT 
    
        c.tpgroup, 
        s.dsregion, 
        SUM(s.qtvolume) as ttlvolume,
        RANK() OVER (PARTITION BY s.dsregion ORDER BY SUM(s.qtvolume) DESC) as ranking
    
    FROM sales s 
        LEFT JOIN channel_group c ON c.cdgroup = s.cdgroup
    GROUP BY c.tpgroup, s.dsregion
)

select spr.* from sales_per_region spr where spr.ranking <=3
''')

In [ ]:
top3.toPandas()

How much sales ($ Volume) each brand (BRAND_NM) achieved per month?


In [ ]:
sales.toPandas()

In [ ]:
sales_volume = spark.sql(

'''
SELECT 
s.dsbrand, s.dsmonth, sum(s.qtvolume) as ttlvolume

FROM sales s 
    GROUP BY  s.dsbrand, s.dsmonth

'''
)

In [ ]:
sales_volume.toPandas()

Which are the lowest brand (BRAND_NM) in sales ($ Volume) for each region (Btlr_Org_LVL_C_Desc)?

In [ ]:
lowest_sales_general = spark.sql('''
SELECT 
s.dsbrand, sum(s.qtvolume) as ttlvolume, s.dsregion, 
RANK() OVER (PARTITION BY s.dsregion ORDER BY  sum(s.qtvolume) DESC) as ranking

FROM sales s
    GROUP BY s.dsbrand, s.dsregion
    ORDER BY s.dsregion ASC, ranking desc
''')

lowest_sales_general.toPandas()

In [ ]:
lowest_volume = spark.sql(
'''
WITH lowest_volume as (
SELECT 
s.dsbrand, sum(s.qtvolume) as ttlvolume, s.dsregion,
RANK() OVER (PARTITION BY s.dsregion ORDER BY sum(s.qtvolume) ASC) as ranking
FROM sales s
    GROUP BY s.dsbrand, s.dsregion)

SELECT l.dsbrand,l.ttlvolume as minvolume, l.dsregion FROM lowest_volume l
  WHERE l.ranking = 1   
    ORDER BY l.dsregion asc
''')


In [ ]:
lowest_volume.toPandas()

In [ ]:
top3.coalesce(1).write.format('csv').save(workdir+'/export/top3')

In [ ]:
sales_volume.coalesce(1).write.format('csv').save(workdir+'/export/sales_volume')

In [ ]:
lowest_volume.coalesce(1).write.format('csv').save(workdir+'/export/lowest_volume')

In [ ]:
!ls -a /home/jovyan/work/export